# Homework - 2
***
**Name**: Ilamvazhuthy Subbiah
***

This assignment is due on Canvas by **5pm on Friday October 5th**. Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your classmates and instructors, but **you must write all code and solutions on your own**, and list any people or sources consulted.

## Boosting - Extra Credit [5-points]
***

In this problem, we slightly modify the AdaBoost algorithm to better explore some properties of the algorithm. Specifically, we no longer normalize the weights on the training examples after each iteration. The modified algorithm, which is set to run for $T$ iterations, is shown in Algorithm I.

Note that in the modified version, the weights associated with the training examples are no longer guaranteed to sum to one after each iteration (and therefore can not be viewed as a "distribution"), but the algorithm is still valid. Let us denote the sum of weights at the start of iteration $t$ by $Z_t = \sum_{i=1}^{n}w_i^{(t)}$. At the start of the first iteration of boosting, $Z_1 = n$. Let us now investigate the behavior of $Z_t$, as a function of t

![image](fig-1.png)

**A:** At the $t^{th}$ iteration, we found a weak classifier that achieves a weighted training error $\epsilon_t$. Show that the choice, $\alpha_t = \frac{1}{2}\log\frac{1 - \epsilon_t}{\epsilon_t}$ is the optimal in the sense that it minimizes $Z_{t+1}$

*Hint: Look at $Z_{t+1}$ as function of $\alpha$ and find the value of $\alpha$ for which the function achieves the minimum. You may also find the following notational shorthand useful:

$$W_t = \sum_{i=1}^{n}w_i^{(t)}(1 - \delta(y_i, h_t(x_i)))$$
$$W_c = \sum_{i=1}^{n}w_i^{(t)}(\delta(y_i, h_t(x_i)))$$

where $W_c$ is the total weight of the points classified correctly by $h_t$ and $W_t$ is the total weight of the misclassified points. $\delta(y, h_t(x)) = 1$ whenever the label predicted by $h_t$ is correct and zero otherwise. The weights here are those available at the start of iteration $t$

$$Z_{t+1}(\alpha_{t}) = W_te^{(\alpha_t)} + W_ce^{-\alpha_t}$$

Fot $Z_{t+1}$ to be the minimum, the derivative of $Z_{t+1}$ with respect to $\alpha$ should be zero. So, we have

$$\frac{dZ_{t+1}}{d\alpha_{t}} =  W_te^{\alpha_t} - W_ce^{-\alpha_t} = 0$$

$$\implies W_te^{\alpha_t} = W_ce^{-\alpha_t}$$

$$\implies e^{2\alpha_t} = \frac{W_c}{W_t}$$

$$\implies 2\alpha_t = \log\frac{W_c}{W_t}$$

And we have
$$ W_c = Z_t(1-\epsilon_t)\quad and \quad W_t = Z_t \epsilon_t$$

$$\implies \alpha_t = \frac{1}{2}\log\frac{Z_t(1-\epsilon_t)}{Z_t\epsilon_t}$$

$$\implies \alpha_t = \frac{1}{2}\log\frac{1-\epsilon_t}{\epsilon_t}$$

Taking the second derivative of $Z_{t+1}$ with respect to $\alpha$, we have

$$\frac{d^2Z_{t+1}}{d{\alpha_{t}}^2} =  W_te^{\alpha_t} + W_ce^{-\alpha_t} > 0$$

Hence it is the minimum.

**B:** Show that the sum of weights $Z_t$ is monotonically decreasing as a function of $t$.

From part A, we have 

$$Z_{t+1} = W_te^{\frac{1}{2}log\frac{W_c}{W_t}} + W_ce^{-\frac{1}{2}log\frac{W_c}{W_t}}$$

$$\implies Z_{t+1} = W_t \sqrt{\frac{W_c}{W_t}} + W_c \sqrt{\frac{W_t}{W_c}}$$

$$\implies Z_{t+1} = 2\sqrt{W_tW_c}$$

so,

$$Z_t - Z_{t+1} = W_c + W_t - 2\sqrt{W_tW_c} = (\sqrt{W_c}-\sqrt{W_t})^2 \geq 0$$

Hence the sum of weights $Z_t$ monotonically decreases as t increases.

## Training Data
***
Please do not change this class

In [2]:
import numpy as np
from sklearn.base import clone

In [3]:
class ThreesAndEights:
    """
    Class to store MNIST data
    """

    def __init__(self, location):

        import pickle, gzip

        # Load the dataset
        f = gzip.open(location, 'rb')

        # Split the data set 
#         X_train, y_train, X_valid, y_valid = pickle.load(f)
        train_set, valid_set, test_set = pickle.load(f)
    
        X_train, y_train = train_set
        X_valid, y_valid = valid_set

        # Extract only 3's and 8's for training set 
        self.X_train = X_train[np.logical_or( y_train==3, y_train == 8), :]
        self.y_train = y_train[np.logical_or( y_train==3, y_train == 8)]
        self.y_train = np.array([1 if y == 8 else -1 for y in self.y_train])
        
        # Shuffle the training data 
        shuff = np.arange(self.X_train.shape[0])
        np.random.shuffle(shuff)
        self.X_train = self.X_train[shuff,:]
        self.y_train = self.y_train[shuff]

        # Extract only 3's and 8's for validation set 
        self.X_valid = X_valid[np.logical_or( y_valid==3, y_valid == 8), :]
        self.y_valid = y_valid[np.logical_or( y_valid==3, y_valid == 8)]
        self.y_valid = np.array([1 if y == 8 else -1 for y in self.y_valid])
        
        f.close()

In [310]:
data = ThreesAndEights("data/mnist.pklz")

Feel free to explore this data and get comfortable with it before proceeding further.

## Bagging
Bootstrap aggregating, also called bagging, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

Given a standard training set $D$ of size n, bagging generates $N$ new training sets $D_i$, roughly each of size n * ratio, by sampling from $D$ uniformly and with replacement. By sampling with replacement, some observations may be repeated in each $D_i$ The $N$ models are fitted using the above $N$ bootstraped samples and combined by averaging the output (for regression) or voting (for classification). 

-Source [Wiki](https://en.wikipedia.org/wiki/Bootstrap_aggregating)

## Implementing Bagging [5-points]
***

We've given you a skeleton of the class `BaggingClassifier` below which will train a classifier based on the decision trees as implemented by sklearn. Your tasks are as follows, please approach step by step to understand the code flow:
* Implement `bootstrap` method which takes in two parameters (`X_train, y_train`) and returns a bootstrapped training set ($D_i$)
* Implement `fit` method which takes in two parameters (`X_train, y_train`) and trains `N` number of base models on different bootstrap samples. You should call `bootstrap` method to get bootstrapped training data for each of your base model
* Implement `voting` method which takes the predictions from learner trained on bootstrapped data points `y_hats` and returns final prediction as per majority rule. In case of ties, return either of the class randomly.
* Implement `predict` method which takes in multiple data points and returns final prediction for each one of those. Please use the `voting` method to reach consensus on final prediction.

In [201]:
from sklearn.tree import DecisionTreeClassifier
import math
from collections import Counter

class BaggingClassifier:
    def __init__(self, ratio = 0.20, N = 20, base=DecisionTreeClassifier(max_depth=4)):
        """
        Create a new BaggingClassifier
        
        Args:
            base (BaseEstimator, optional): Sklearn implementation of decision tree
            ratio: ratio of number of data points in subsampled data to the actual training data
            N: number of base estimator in the ensemble
        
        Attributes:
            base (estimator): Sklearn implementation of decision tree
            N: Number of decision trees
            learners: List of models trained on bootstrapped data sample
        """
        
        assert ratio <= 1.0, "Cannot have ratio greater than one"
        self.base = base
        self.ratio = ratio
        self.N = N
        self.learners = []
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        #TODO: Implement functionality to fit models on the bootstrapped samples
        # cloning sklearn models:
        # from sklearn.base import clone
        # h = clone(self.base)
        
        for i in range(self.N):
            h = clone(self.base)
            X_bstrap, y_bstrap = self.bootstrap(X_train, y_train)
            h.fit(X_bstrap, y_bstrap)
            self.learners.append(h)
        
    def bootstrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        random_list = [i for i in range(0, len(X_train))]
        np.random.shuffle(random_list)
        
        X_bstrap = np.array([X_train[random_list[i]] for i in range(0, math.floor(len(X_train)*self.ratio))])
        y_bstrap = np.array([y_train[random_list[i]] for i in range(0, math.floor(len(y_train)*self.ratio))])
        
        return X_bstrap, y_bstrap
        
    
    def predict(self, X):
        """
        BaggingClassifier prediction for data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        #TODO: Using the individual classifiers trained predict the final prediction using voting mechanism
        
        # Get the prediction from individual classifiers
        y_hats = []
        for h in self.learners:
            y_hats.append(h.predict(X))
        y_hat = np.array(y_hats).T
        
        # Get the majority by voting
        y_predict = []
        for i in range(0, len(X)):
            y_predict.append(self.voting(y_hat[i]))
        
        return y_predict
            
    
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the 
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        return Counter(y_hats).most_common(1)[0][0]

## BaggingClassifier for Handwritten Digit Recognition [5-points]
***

After you've successfully completed `BaggingClassifier` find the optimal values of `ratio`, `N` and `depth` using k-fold cross validation. You are allowed to use sklearn library to split your training data in folds. Use the data from `ThreesAndEights` class initialized variable `data`. 

Justify why those values are optimal.

Report accuracy on the validation data using the optimal parameter values.

In [322]:
from sklearn.model_selection import KFold
kf = KFold(3)
ratio = [0.1, 0.2, 0.4, 0.8]
N = [5, 10, 20, 40]
depth = [5, 10, 20]
max_accuracy = 0
max_N = 0
max_r =0
max_d = 0
for r in ratio:
    for n in N:
        for d in depth: 
            accuracy = []
            for train, test in kf.split(data.X_train):
                X_train, X_test, y_train, y_test = data.X_train[train], data.X_train[test], data.y_train[train], data.y_train[test]         
                bagging_classifier = BaggingClassifier(r, n, DecisionTreeClassifier(max_depth=d) )
                bagging_classifier.fit(X_train, y_train)
                y_result = bagging_classifier.predict(X_test)
                count = 0
                for i in range(len(y_test)):
                    if y_test[i] == y_result[i]:
                        count += 1
                #print("Accuracy on validation set = ", count/len(y_test))
                accuracy.append(count/len(y_test))
                avg_accuracy = sum(accuracy)/len(accuracy)
                if  avg_accuracy > max_accuracy:
                    max_accuracy = avg_accuracy
                    max_r = r
                    max_N = n
                    max_d = d
print("The optimal ratio is", max_r)
print("The optimal N is", max_N)
print("The optimal depth is", max_d)
print("Average Accuracy on all folds with the optimal parameters is", max_accuracy )

The optimal ratio is 0.4
The optimal N is 40
The optimal depth is 20
Average Accuracy on all folds with the optimal parameters is 0.9741528011789646


In [332]:
bagging_classifier = BaggingClassifier(0.4, 40, DecisionTreeClassifier(max_depth=20))
bagging_classifier.fit(data.X_train, data.y_train)
y_result = bagging_classifier.predict(data.X_valid)
count = 0
for i in range(len(data.y_valid)):
    if data.y_valid[i] == y_result[i]:
        count += 1
accuracy = count/len(data.y_valid)
print("The accuracy on the validation data using the optimal parameters is", accuracy)

The accuracy on the validation data using the optimal parameters is 0.9789112309955861


The parameters are optimal beacause they hit the sweet spot between overfitting and underfitting.

# Random Decision Tree [10-points]

In this assignment you are going to implement a random decision tree using random vector method as discussed in the lecture.

Best split: One that achieves maximum reduction in gini index across multiple candidate splits. (decided by `candidate_splits` attribute of the class `RandomDecisionTree`)

Use `TreeNode` class as node abstraction to build the tree

You are allowed to add new attributes in the `TreeNode` and `RandomDecisionTree` class - if that helps.

Your tasks are as follows:
* Implement `gini_index` method which takes in class labels as parameter and returns the gini impurity as measure of uncertainty

* Implement `majority` method which picks the most frequent class label. In case of tie return any random class label

* Implement `find_best_split` method which finds the random vector/hyperplane which causes most reduction in the gini index. 

* Implement `build_tree` method which uses `find_best_split` method to get the best random split vector for current set of training points. This vector partitions the training points into two sets, and you should call `build_tree` method on two partitioned sets and build left subtree and right subtree. Use `TreeNode` as abstraction for a node.

> The method calls itself recursively to the generate left and right subtree till the point either `max_depth` is reached or no good random split is found.  When either of two cases is encountered, you should make that node as leaf and identify the label for that leaf to be the most frequent class (use `majority` method). Go through lecture slides for better understanding

* Implement `predict` method which takes in multiple data points and returns final prediction for each one of those using the tree built. (`root` attribute of the class)

In [303]:
class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.isLeaf = False
        self.label = None
        self.split_vector = None

    def getLabel(self):
        if not self.isLeaf:
            raise Exception("Should not to do getLabel on a non-leaf node")
        return self.label
    
class RandomDecisionTree:
            
    def __init__(self, candidate_splits = 100, depth = 10):
        """
        Args:
            candidate_splits (int) : number of random decision splits to test
            depth (int) : maximum depth of the random decision tree
        """
        self.candidate_splits = candidate_splits
        self.depth = depth
        self.root = None
    
    def fit(self, X_train, y_train):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
        """
        self.root = self.build_tree(X_train[:], y_train[:], 0)
        return self
        
    def build_tree(self, X_train, y_train, height):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
        """
        node = TreeNode()
        node.split_vector = self.find_best_split(X_train, y_train)
        X_left =[]
        X_right = []
        y_left = []
        y_right =[]
        if node.split_vector is not None and height < self.depth:
            y_split = np.dot(X_train, node.split_vector)
            for i in range(0, len(y_split)):
                if y_split[i] <=0:
                    X_left.append(X_train[i])
                    y_left.append(y_train[i])
                else:
                    X_right.append(X_train[i])
                    y_right.append(y_train[i])
            node.left = self.build_tree(X_left, y_left, height+1)
            node.right = self.build_tree(X_right, y_right, height+1)
        else:
            node.isLeaf = True
            node.label = self.majority(y_train)
        return node
    
    def find_best_split(self, X_train, y_train):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
        """
        split_vector = None
        gain = 0
        for j in range(0, self.candidate_splits):
            vec = np.zeros(len(X_train[0]))
            
            for i in range(len(X_train[0])):
                vec[i] = (np.random.random_sample())*2 -1
            y_left = []
            y_right = []

            y_split = np.dot(X_train, vec)
            for i in range(0,len(y_split)):
                if y_split[i] <=0:
                    y_left.append(y_train[i])
                else:
                    y_right.append(y_train[i])

            gain_temp = self.gini_index(y_train) - ((self.gini_index(y_left))*len(y_left) + (self.gini_index(y_right))*len(y_right))/len(y_train)
            if gain_temp > gain:
                split_vector = vec
                gain = gain_temp
        
        return split_vector
            
        
    def gini_index(self, y):
        """
        Args:
            y (ndarray): [n_samples] ndarray of data
        """
        counts = {}
        for label in y:
            counts[label] = counts.get(label, 0) + 1
        
        gini_ix = 1
        for label in counts:
            gini_ix -= (counts[label]/len(y))**2
            
        return gini_ix
        
    
    def majority(self, y):
        """
        Return the major class in ndarray y
        """
        return Counter(y).most_common(1)[0][0]
    
    def predict(self, X):
        """
        BaggingClassifier prediction for new data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        yhat = []
        for i in range(0, len(X)):
            node = self.root
            while(not node.isLeaf):
                split_vector = node.split_vector
                y_split = np.dot(split_vector, X[i])
                if y_split <= 0:
                    node = node.left
                    split_vector = node.split_vector
                else:
                    node = node.right
                    split_vector = node.split_vector
            yhat.append(node.getLabel())
        return np.array(yhat)

## RandomDecisionTree for Handwritten Digit Recognition

After you've successfully completed `RandomDecisionTree`, and train using the default values in the constructor and report accuracy on the test set. Use the data from `ThreesAndEights` class initialized variable `data` 

In [306]:
tree = RandomDecisionTree()
tree.fit(data.X_train, data.y_train)

In [307]:
y_tree = tree.predict(data.X_valid)
count = 0
for i in range(len(data.y_valid)):
    if data.y_valid[i] == y_tree[i]:
        count += 1
print("Accuracy on validation set = ", count/len(data.y_valid))

Accuracy on validation set =  0.8930848455125061


# Random Forest [5-points]
Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set.

Random forest trains random decision trees on bootstrapped training points. Thus, you can implementation of methods (`bootstrap`, `predict`) from `BaggingClassifier` class directly. Only difference being, you have to use the `RandomDecisionTree` as base which you implemented previously instead of sklearn's implementation of `DecisionTreeClassifier`). Implement the `fit` method in the class below accordingly.

In [308]:
class RandomForest(BaggingClassifier):
    def __init__(self, ratio = 0.20, N = 20, max_depth = 10, candidate_splits = 100):
        self.ratio = ratio
        self.N = N
        self.learners = []
        self.candidate_splits = candidate_splits
        self.max_depth = max_depth
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        for i in range(self.N):
            h = RandomDecisionTree()
            X_bstrap, y_bstrap = self.bootstrap(X_train, y_train)
            h.fit(X_bstrap, y_bstrap)
            self.learners.append(h)
        

## RandomForest for Handwritten Digit Recognition [5-points]
***

After you've successfully completed `RandomForest` find the optimal values of `ratio`, `N`, `candidate_splits` and `depth` using k-fold cross validation on. Feel free to use sklearn library to split your training data. Use the data from `ThreesAndEights` class intialized variable `data`. 

Justify why those values are optimal.

Report best accuracy on the testing data using those optimal parameter values.

In [309]:
kf = KFold(5)
for train, test in kf.split(data.X_train):
    X_train, X_test, y_train, y_test = data.X_train[train], data.X_train[test], data.y_train[train], data.y_train[test]
    bagging_classifier = RandomForest()
    bagging_classifier.fit(X_train, y_train)
    y_result = bagging_classifier.predict(X_test)
    count = 0
    for i in range(len(y_test)):
        if y_test[i] == y_result[i]:
            count += 1
    print("Accuracy on validation set = ", count/len(y_test))

Accuracy on validation set =  0.9658119658119658
Accuracy on validation set =  0.9507290095525389
Accuracy on validation set =  0.9552538964303671
Accuracy on validation set =  0.9456740442655935
Accuracy on validation set =  0.954728370221328


The above accuracies are on the test set. I did not have the time change the print statement and run it again.

In [336]:
ratio = [0.2, 0.4, 0.8]
N = [10, 20, 40]
depth = [5, 10]
splits = [20, 50, 100]
max_accuracy = 0
max_N = 0
max_r =0
max_d = 0
max_split = 0
kf = KFold(3)
for r in ratio:
    for n in N:
        for d in depth:
            for s in splits:
                accuracy = []
                for train, test in kf.split(data.X_train[:100]):
                    X_train, X_test, y_train, y_test = data.X_train[train], data.X_train[test], data.y_train[train], data.y_train[test]
                    bagging_classifier = RandomForest(r, n, d, s)
                    bagging_classifier.fit(X_train, y_train)
                    y_result = bagging_classifier.predict(X_test)
                    count = 0
                    for i in range(len(y_test)):
                        if y_test[i] == y_result[i]:
                            count += 1
                    accuracy.append(count/len(y_test))
                    avg_accuracy = sum(accuracy)/len(accuracy)
                    if  avg_accuracy > max_accuracy:
                        max_accuracy = avg_accuracy
                        max_r = r
                        max_N = n
                        max_d = d
                        max_split = s
print("The optimal ratio is", max_r)
print("The optimal N is", max_N)
print("The optimal depth is", max_d)
print("The optimal candidate splits is", max_split)
print("Average Accuracy on all folds with the optimal parameters is", max_accuracy )

The optimal ratio is 0.4
The optimal N is 10
The optimal depth is 10
The optimal candidate splits is 100
Average Accuracy on all folds with the optimal parameters is 0.9705882352941176


In [ ]:
rf_classifier = RandomForest(0.4, 20, 10, 100)
rf_classifier.fit(data.X_train, data.y_train)
y_result = rf_classifier.predict(data.X_valid)
count = 0
for i in range(len(data.y_valid)):
    if data.y_valid[i] == y_result[i]:
        count += 1
accuracy = count/len(data.y_valid)
print("The accuracy on the validation data using the optimal parameters is", accuracy)

The parameters are optimal beacause the variance is low and they hit the sweet spot between overfitting and underfitting.